## Meta matching v1.0
This jupyter notebook demonstrates you how to load and use meta-matching algorthm. In this demonstration, we performed Meta-matching (DNN) stacking with 100 example subjects.

Package needed (and version this jupyter notebook tested):
* Numpy (1.24.4)
* Scipy (1.10.1)
* PyTorch (2.0.0)
* Scikit-learn (1.3.2)

### Step 0. Setup
Please modify the `path_repo` below to your repo position:

In [1]:
path_repo = "../" #'/home/the/deepGround/code/2002/Meta_matching_models/'

In [2]:
# initialization and random seed set

import os
import sys
import random
import scipy
import torch
import sklearn
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

import warnings
warnings.filterwarnings("ignore")

Please modify the gpu number here if you want to use different gpu. If the gpu you assigned not availiable, it will assign to cpu

In [3]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

### Step 1. load data
Load the example data that we provided, it contains 
* Example input functional connectivity (FC) `x` with size of (100, 87571)
    * 100 is number of subjects
    * 87571 is flatten vector of 419 by 419 FC (419*418/2=87571)
* Example output phenotypes `y` with size of (100, 3)
    * 3 is number of phenotypes.

In [4]:
path_v1 = os.path.join(path_repo, 'v1.0')
path_data = os.path.join(path_repo, 'data')
npz = os.path.join(path_data, 'meta_matching_example_data.npz')
npz = np.load(npz)
print(npz.files)
x_input = npz['x']
y_input = npz['y']
print(x_input.shape, y_input.shape)

['x', 'y']
(100, 87571) (100, 3)


### Step 2. Split data
Here, we also split 100 subjects to 80/20, where 80 for training, and 20 for test.

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_input, y_input, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(80, 87571) (20, 87571) (80, 3) (20, 3)


After data split, we z-normalize the label (y) of the training and testing set, using mean and std of the training set.

In [6]:
def z_norm(y_train, y_test):
    # subtract mean of y of the training set
    t_mu = np.nanmean(y_train, axis=0, keepdims=True)
    y_train = y_train - t_mu
    y_test = y_test - t_mu

    # divide std of y of the training set
    t_sigma = np.nanstd(y_train, axis=0)
    if y_train.ndim == 2:
        t_sigma_d = t_sigma[np.newaxis, :]
    else:
        t_sigma_d = t_sigma
        if t_sigma == 0:
            return y_train, y_test
    y_train = y_train / t_sigma_d
    y_test = y_test / t_sigma_d
    return y_train, y_test

y_train, y_test = z_norm(y_train, y_test)

### Step 3. Prepare data for PyTorch model
Then we prepare data for DNN model, we will input both the `x_train` and `x_test` into the model to get the predicted phenotypes. 

For meta-matching (DNN) stacking, we do not need real phenotype for the DNN model, I created all zeros `y_dummy` just for function requirement. In some other cases, like meta-matching (DNN) finetuning, you need to use real phenotype data here.

In [7]:
from torch.utils.data import DataLoader
sys.path.append(os.path.join(path_repo, "utils"))
from CBIG_model_pytorch import multi_task_dataset

batch_size = 16
y_dummy = np.zeros(y_train.shape)
dset_train = multi_task_dataset(x_train, y_dummy, True)
trainLoader = DataLoader(dset_train,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=1)

y_dummy = np.zeros(y_test.shape)
dset_test = multi_task_dataset(x_test, y_dummy, True)
testLoader = DataLoader(dset_test,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=1)

### Step 4. load model
Here we load the meta-matching model saved, it is a DNN that takes FC as input and output 67 phenotypes prediction trained on 67 UK Biobank phenotypes

In [8]:
path_model_weight = os.path.join(path_v1, 'meta_matching_v1.0_model.pkl_torch')  
net = torch.load(path_model_weight, map_location=device)
net.train(False)
print(net)

dnn_4l(
  (fc1): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=87571, out_features=118, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(118, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc2): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=118, out_features=445, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(445, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc3): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=445, out_features=353, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(353, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc4): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=353, out_features=67, bias=True)
  )
)


### Step 5. DNN model predict
Here we apply the DNN trained on 67 UK Biobank phenotypes to predict the 67 phenotypes on `x_train` and `x_test`. We will get the predicted 67 phenotypes on both 80 training subjects and 20 test subjects.

In [9]:
y_train_pred = np.zeros((0, 67))
for (x, _) in trainLoader:
    x= x.to(device)
    outputs = net(x)
    y_train_pred = np.concatenate((y_train_pred, outputs.data.cpu().numpy()), axis=0)
print(y_train_pred.shape, '\n', y_train_pred)

y_test_pred = np.zeros((0, 67))
for (x, _) in testLoader:
    x= x.to(device)
    outputs = net(x)
    y_test_pred = np.concatenate((y_test_pred, outputs.data.cpu().numpy()), axis=0)
print(y_test_pred.shape, '\n', y_test_pred)

(80, 67) 
 [[-0.17227212  0.00659018 -0.30208847 ... -0.15928994 -0.38077325
   0.03831919]
 [ 0.0840143  -0.2012696   0.62567234 ...  0.14059748 -0.0080429
   0.4631604 ]
 [ 0.13499442  0.14025491 -0.51653886 ...  0.01679245 -0.21242383
  -0.15089568]
 ...
 [ 0.14017352 -0.13507903  0.06178959 ...  0.14889181  0.00375715
   0.00102939]
 [-0.00132074 -0.20964417  0.35207599 ...  0.17323518 -0.1256841
  -0.07328446]
 [-0.14659923  0.09759457 -0.24224234 ... -0.07367411 -0.56735659
  -0.06110727]]
(20, 67) 
 [[-0.07091516 -0.20356585  0.30995744 ...  0.17636669 -0.36347184
  -0.13356218]
 [-0.01349915 -0.13479879  0.26539761 ...  0.22685154  0.03282836
  -0.25569448]
 [ 0.03873897 -0.2232421   0.34538248 ...  0.2163894  -0.10278906
  -0.06968378]
 ...
 [ 0.01483439 -0.27222607  0.38793972 ...  0.13031584 -0.24093717
   0.35386068]
 [-0.11500867 -0.14910071  0.18525358 ...  0.11365267 -0.32046539
  -0.13270387]
 [ 0.13167746  0.13831377 -0.34880367 ...  0.00669172  0.02045992
  -0.1248267

### Step 6. Stacking
Perform stacking with `y_train_pred`, `y_test_pred`, `y_train`, where we use the prediction of 80 subjects `y_train_pred` (input) and real data `y_train` (output) to train the stacking model (you can either use all 67 source phenotypes for stacking, or select top K source phenotypes relevant to the target phenotype, like we mentioned in our paper; it turns out that these 2 ways achieves similar performances), then we applied the model to `y_test_pred` to get final prediction of 3 phenotypes on 20 subjects.

#### Hyperparameter Tuning 
In `stacking()` function, we set the range of `alpha` as `[0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20]`. You are weclomed to modify the range of `alpha` to get better performance on your own data.

In [10]:
from CBIG_model_pytorch import stacking
y_test_final=np.zeros((y_test_pred.shape[0], y_train.shape[1]))
for i in range(y_train.shape[1]):
    # For each test phenotype, perform stacking by developing a KRR model
    y_test_temp, _ = stacking(y_train_pred, y_test_pred, y_train[:,i].view(), [0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20])
    y_test_final[:,i] = y_test_temp.flatten()
print(y_test_final.shape, '\n', y_test_final)

(20, 3) 
 [[ 0.30858318  0.31614136  0.25616881]
 [ 0.33097868  0.27920972  0.22188792]
 [ 0.3022932   0.27643389 -0.21590371]
 [-0.30724698 -0.25114863 -0.09360205]
 [ 0.29601656  0.23809381 -0.08588831]
 [ 0.03028216  0.06092177 -0.42269768]
 [ 0.05079768  0.06213083 -0.00315667]
 [-0.20391299 -0.12172257 -0.28335082]
 [-0.23192853 -0.24666243 -0.11799798]
 [ 0.1687797   0.12687031 -0.21025245]
 [-0.25755716 -0.26978648 -0.12037667]
 [ 0.11587517  0.13568884 -0.404235  ]
 [ 0.33310192  0.29047687  0.4093741 ]
 [ 0.28479089  0.24948804  0.43535864]
 [ 0.19712837  0.17144201  0.18689087]
 [ 0.06142156  0.08956782 -0.34495887]
 [-0.2453573  -0.19865241  0.21710906]
 [ 0.03216124  0.04582251 -1.02856156]
 [ 0.33752271  0.31074357  0.68634819]
 [-0.14581154 -0.17190664  0.18438439]]


### Step 7. Evaluation
Evaluate the prediction performance. Note that we didn't reverse the z-normalization which previously applied on `y_train` and `y_test`. This is because the metric (Pearson correlation) would not be affected by z-normalization. If you are predicting the phenotypes for practical use, you are recommended to reverse the z-normalization 

In [11]:
from scipy.stats.stats import pearsonr
corr = np.zeros((y_train.shape[1]))
for i in range(y_train.shape[1]):
    corr[i] = pearsonr(y_test_final[:, i], y_test[:, i])[0]
print(corr)

[0.40592904 0.40829964 0.21940492]


### Step 8. Haufe transform predictive network features (PNFs) computation
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each element of FC on the 80 training subjects. The final PNF is in shape of (87571, 3), where 87571 is number of 419 by 419 FC elements, and 3 is number of phenotypes.

In [12]:
from CBIG_model_pytorch import covariance_rowwise

y_train_haufe, _ = stacking(y_train_pred, y_train_pred, y_train)
print(y_train_haufe.shape)
cov = covariance_rowwise(x_train, y_train_haufe)
print(cov, '\n', cov.shape)

(80, 3)
[[ 0.00732235  0.00713506  0.00118967]
 [ 0.01078571  0.01010187  0.00199967]
 [ 0.00441558  0.00467486 -0.00158594]
 ...
 [ 0.01436255  0.01190753  0.00724397]
 [ 0.01585564  0.01320002  0.00776838]
 [ 0.0149424   0.01306513  0.00628666]] 
 (87571, 3)


### Step 9. Haufe transform predictive network features (PNFs) computation for training phenotypes
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each training phenotypes on the 80 training subjects. The final PNF is in shape of (67, 3), where 67 is number of training phenotypes, and 3 is number of phenotypes.

In [13]:
from CBIG_model_pytorch import covariance_rowwise

cov = covariance_rowwise(y_train_pred, y_train_haufe)
print(cov, '\n', cov.shape)

[[ 1.62112542e-03 -7.29563194e-04  4.11851982e-04]
 [-2.88318801e-02 -2.60968040e-02 -5.33670361e-03]
 [ 8.37645040e-02  7.14207328e-02  3.11697183e-02]
 [ 4.08589351e-03  4.56096290e-03  6.94921181e-03]
 [ 5.16990796e-03  3.27474343e-03 -9.38602973e-03]
 [-2.38979929e-02 -2.18224532e-02 -2.05118513e-02]
 [ 8.63389902e-03  7.05434234e-03  1.16107905e-02]
 [ 9.56680719e-03  8.05383664e-03  1.34339202e-02]
 [ 6.66705145e-02  5.45499531e-02  2.43841043e-02]
 [ 3.18675326e-03  3.99848474e-04 -5.76661315e-03]
 [ 3.63114426e-02  2.91520766e-02  1.61062252e-02]
 [-3.38178361e-02 -3.20809520e-02 -1.43090071e-02]
 [-1.70428594e-02 -1.27157029e-02 -1.28376742e-02]
 [ 5.94135870e-02  4.81916649e-02  5.39425676e-02]
 [ 2.02485478e-02  1.51641613e-02  1.97555790e-02]
 [ 2.07869438e-01  1.81910344e-01  8.34806334e-02]
 [-5.58790195e-02 -4.52913699e-02 -3.14647489e-02]
 [-3.17880138e-02 -1.88238709e-02 -2.19619521e-02]
 [-5.72028636e-02 -4.55577413e-02 -4.35474789e-02]
 [-3.18404322e-03 -3.18153740e-